In [ ]:
import pyarrow.parquet as pq
from pyiceberg.catalog import load_catalog

In [ ]:
! curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o ../data/yellow_tripdata_2023-01.parquet

In [ ]:
df = pq.read_table("../data/yellow_tripdata_2023-01.parquet")

In [ ]:
print(df)

In [ ]:
catalog = load_catalog(
    name="hive",
    **{
        "type": "hive",
        "uri": "thrift://hive-metastore:9083",
        "warehouse": "s3a://iceberg/lakehouse",
        "s3.endpoint": "http://minio:9000",
        "s3.access-key-id": "minio",
        "s3.secret-access-key": "minio123",
        "s3.path-style-access": "true",
        "s3.region": "us-east-1",
    }
)

In [ ]:
catalog.create_namespace("bronze")

In [ ]:
table = catalog.create_table(
    "bronze.taxi_dataset",
    schema=df.schema,
    location="s3a://iceberg/lakehouse/bronze/iceberg_nyc_taxi"
)

In [ ]:
table.append(df)
len(table.scan().to_arrow())